In [1]:
import argparse
import os
import random,numpy
os.environ['CUBLAS_WORKSPACE_CONFIG'] = ':4096:8'

In [2]:
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.nn.functional as F
import torch.utils.data
import torchvision
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
import torchvision.transforms.functional as RF
from PIL import Image
import numpy as np
import random,cv2,pandas,os,numpy
import matplotlib.pyplot as plt
import matplotlib.animation as animation

In [3]:
seed = 999
print("Random Seed: ", seed)
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)  # if you are using multi-GPU.
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.use_deterministic_algorithms(True)

workers = 2
nz = 100
beta1 = 0.5
lr = 0.0001
ngpu=1
ngf,nc = 3,3
ndf = 64

device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")

Random Seed:  999


In [4]:
def weights_init(m):
    if isinstance(m, nn.Linear):
        nn.init.xavier_normal_(m.weight.data, gain=0.02)
        
class Discriminator(nn.Module):
    def __init__(self):
        super().__init__()
        self.rafire_0 = nn.LSTM(input_size=1024, hidden_size=2024, num_layers=1, batch_first=True)
        self.rafire = nn.Sequential(
            
            nn.Linear(2024, 1524),
            nn.Dropout(p=0.35),
            nn.SiLU(),
            
            nn.Linear(1524, 1124),
            nn.Dropout(p=0.30),
            nn.SiLU(),
            
            nn.Linear(1124, 924),
            nn.Dropout(p=0.30),
            nn.SiLU(),
            
            nn.Linear(924, 824),
            nn.Dropout(p=0.25),
            nn.SiLU(),
            
            nn.Linear(824, 724),
            nn.Dropout(p=0.25),
            nn.SiLU(),
            
            nn.Linear(724, 624),
            nn.Dropout(p=0.2),
            nn.SiLU(),
            
            nn.Linear(624, 524),
            nn.Dropout(p=0.2),
            nn.SiLU(),
            
            nn.Linear(524, 424),
            nn.Dropout(p=0.15),
            nn.SiLU(),
            
            nn.Linear(424, 324),
            nn.Dropout(p=0.1),
            nn.SiLU(),
            
            nn.Linear(324, 283),
            nn.SiLU()
        )

    def forward(self,x):
        x, _=self.rafire_0(x)
        return self.rafire(x)

In [5]:
netD = Discriminator().to(device)
if (device.type == 'cuda') and (ngpu > 1):
    netD = nn.DataParallel(netD, list(range(ngpu)))
netD.apply(weights_init)

#netD.load_state_dict(torch.load(f"/kaggle/input/radire-model/result_rf_wl/4.053906650369754e-06.pth",map_location=torch.device('cpu'),weights_only=False))

Discriminator(
  (rafire_0): LSTM(1024, 2024, batch_first=True)
  (rafire): Sequential(
    (0): Linear(in_features=2024, out_features=1524, bias=True)
    (1): Dropout(p=0.35, inplace=False)
    (2): SiLU()
    (3): Linear(in_features=1524, out_features=1124, bias=True)
    (4): Dropout(p=0.3, inplace=False)
    (5): SiLU()
    (6): Linear(in_features=1124, out_features=924, bias=True)
    (7): Dropout(p=0.3, inplace=False)
    (8): SiLU()
    (9): Linear(in_features=924, out_features=824, bias=True)
    (10): Dropout(p=0.25, inplace=False)
    (11): SiLU()
    (12): Linear(in_features=824, out_features=724, bias=True)
    (13): Dropout(p=0.25, inplace=False)
    (14): SiLU()
    (15): Linear(in_features=724, out_features=624, bias=True)
    (16): Dropout(p=0.2, inplace=False)
    (17): SiLU()
    (18): Linear(in_features=624, out_features=524, bias=True)
    (19): Dropout(p=0.2, inplace=False)
    (20): SiLU()
    (21): Linear(in_features=524, out_features=424, bias=True)
    (22): D

In [6]:
criterion,img_devil = nn.MSELoss(),0
fixed_noise = torch.randn(1, nz, 1, 1, device=device)

optimizerD = optim.AdamW(netD.parameters(), lr=lr,betas=(beta1, 0.999))
schedulerD=torch.optim.lr_scheduler.ExponentialLR(optimizerD, gamma=0.86)

In [7]:
train_label_=pandas.read_csv("/kaggle/input/ariel-data-challenge-2024/train_labels.csv")
train_data_,train_label=[],[]

batch_size=10
batch_d,batch_l=[],[]
for i in train_label_['planet_id']:
    train_data=numpy.load(f"/kaggle/input/rafire-ariel/train_rafire/{i}/train_data.npy",allow_pickle='TRUE')
    #if len(batch_d)==batch_size:
        #train_data_.append(numpy.array(batch_d)),train_label.append(numpy.array(batch_l))
        #batch_d,batch_l=[],[]
    #batch_d.append()
    #print(train_data.shape)
    #print((numpy.array([numpy.array(train_label_[train_label_['planet_id']==int(i)]).reshape(-1)[1::] for a in range(2700)]).shape))
    train_data_.append(numpy.array([j.reshape(-1) for j in train_data]))
    train_label.append(numpy.array([numpy.array(train_label_[train_label_['planet_id']==int(i)]).reshape(-1)[1::] for a in range(1000)]))

    #batch_l.append(numpy.array(train_label_[train_label_['planet_id']==int(i)]).reshape(-1)[1::])
    
train_data,train_label=torch.from_numpy(numpy.array(train_data_)).to(device),torch.from_numpy(numpy.array(train_label)).to(device)
train_data.shape,train_label.shape

(torch.Size([673, 1000, 1024]), torch.Size([673, 1000, 283]))

In [8]:
train_label_,z_w_=pandas.read_csv("/kaggle/input/ariel-data-challenge-2024/train_labels.csv"),[]
high_rf,i,j_r,k=1000,0,0,0
if os.path.exists(f"/kaggle/working/result_rf_wl")==False:
    os.mkdir(f"/kaggle/working/result_rf_wl")

while(True):
    optimizerD.zero_grad()
    for data in range(len(train_data)):
        #print(data)
        output = abs(netD(train_data[data].float()))#-0.0015)
        errD_real = criterion(output.float(),train_label[data].float())
        errD_real.backward()
        optimizerD.step()
        #print(f"LOSS_wl:{errD_real.item()}")
    
    print(f"EPOCH:{i} LOSS_wl:{errD_real.item()}")
    if len(z_w_)>=3:
        if len([True for i in range(1,4) if z_w_[len(z_w_)-i]<=z_w_[len(z_w_)-3] and z_w_[len(z_w_)-i]>=z_w_[len(z_w_)-4]])==3:
            z_w_,j_r=[],0
            print(f"lr_br:{optimizerD.param_groups[0]['lr']}".upper())
            schedulerD.step()
            print(f"lr_up:{optimizerD.param_groups[0]['lr']}".upper())
    z_w_.append(errD_real.item())
    if errD_real.item()<high_rf:
            high_rf=errD_real.item()
            torch.save(netD.state_dict(),f'/kaggle/working/result_rf_wl/{errD_real.item()}.pth')
    
    """if k==10:
        k=0
        if errD_real.item()<high_rf:
            high_rf=errD_real.item()
            torch.save(netD.state_dict(),f'/kaggle/working/result_rf_wl/{errD_real.item()}.pth')
        for i_ in train_label_['planet_id'][:2]:
            real_cpu=torch.from_numpy(train_data_z[i_].reshape(-1)).to(device)
            output = abs(netD(real_cpu.float()))#-0.0015)
            print(f"PREDICTED: {output[:5].cpu().detach().numpy()}")

            label=numpy.array(train_label_[train_label_['planet_id']==int(i_)]).reshape(-1)[1::]
            print(f"TRUE: {label[:5]}")"""

    if i>=20:
        break
    i+=1
    j_r+=1
    k+=1

EPOCH:0 LOSS_wl:0.0007917711627669632
EPOCH:1 LOSS_wl:2.3681623133597896e-06
EPOCH:2 LOSS_wl:2.3681623133597896e-06
EPOCH:3 LOSS_wl:2.3681623133597896e-06
LR_BR:0.0001
LR_UP:8.6E-05
EPOCH:4 LOSS_wl:2.3681623133597896e-06
EPOCH:5 LOSS_wl:2.3681623133597896e-06
EPOCH:6 LOSS_wl:2.3681623133597896e-06
LR_BR:8.6E-05
LR_UP:7.396E-05
EPOCH:7 LOSS_wl:2.3681623133597896e-06
EPOCH:8 LOSS_wl:2.3681623133597896e-06
EPOCH:9 LOSS_wl:2.3681623133597896e-06
LR_BR:7.396E-05
LR_UP:6.36056E-05
EPOCH:10 LOSS_wl:2.3681623133597896e-06
EPOCH:11 LOSS_wl:2.3681623133597896e-06
EPOCH:12 LOSS_wl:2.3681623133597896e-06
LR_BR:6.36056E-05
LR_UP:5.4700816E-05
EPOCH:13 LOSS_wl:2.3681623133597896e-06
EPOCH:14 LOSS_wl:2.3681623133597896e-06
EPOCH:15 LOSS_wl:2.3681623133597896e-06
LR_BR:5.4700816E-05
LR_UP:4.704270176E-05
EPOCH:16 LOSS_wl:2.3681623133597896e-06
EPOCH:17 LOSS_wl:2.3681623133597896e-06
EPOCH:18 LOSS_wl:2.3681623133597896e-06
LR_BR:4.704270176E-05
LR_UP:4.04567235136E-05
EPOCH:19 LOSS_wl:2.368162313359789